In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trying").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 11:12:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [7]:
import pp_events as pp

In [8]:
events_shot = pp.preprocessing(events)

In [9]:
events_shot.show()

+--------------------+---------------+---------------+---------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|                  id|shot_location_x|shot_location_y|player_id|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|
+--------------------+---------------+---------------+---------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|c7168289-8ba6-427...|          111.6|           34.7|  30111.0|          9.9323| 39.113426|                  0|             1|              1|              0|              0|             0|                   0|       0.05332383|       Off T|   0|
|aa63a13

In [6]:
columns_t = ["shot_location_x", "shot_location_y",
             'distance_to_goal','shot_angle','preferred_foot_shot',
             'under_pressure','shot_statsbomb_xg','goal']

In [7]:
events_shot = events_shot.select(columns_t)

In [8]:
events_shot.show()

+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+-----+
|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_statsbomb_xg| goal|
+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+-----+
|          106.2|           36.8|         14.1662| 30.870586|               true|          NULL|       0.15133068|false|
|          114.8|           44.1|          6.6219|  56.19886|              false|          true|       0.14169177|false|
|           94.8|           24.9|         29.3777| 13.387468|               true|          NULL|      0.039566465|false|
|          111.1|           42.0|           9.122| 46.651226|              false|          NULL|       0.26291034|false|
|           98.6|           25.2|         26.0192| 14.520617|               true|          NULL|       0.02977089|false|
|          106.1|           32.5

In [9]:
# Columns to convert to int
cols = ['preferred_foot_shot','under_pressure','goal']
events_shot = pp.bool_to_int(events_shot,cols)

In [10]:
events_shot.show()

+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+----+
|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_statsbomb_xg|goal|
+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+----+
|          111.6|           34.7|          9.9323| 39.113426|                  0|             1|       0.05332383|   0|
|          115.6|           43.0|          5.3254| 70.651955|                  0|             1|       0.15510525|   0|
|           94.3|           46.3|         26.4609| 16.725826|                  1|             0|      0.041628048|   0|
|          107.9|           33.1|         13.9291|  28.53554|                  0|             0|      0.033826087|   0|
|          107.5|           44.2|         13.1867|  32.34822|                  0|             0|       0.02513541|   0|
|          113.4|           38.6|       

In [11]:
# our goal is to come up with a measure of the quality of the chance to score a goal, an xG model
# therefore we will use the goal column as a target value in the training, and the shot_statsbomb_xg as the y to evaluate the model

In [12]:
#features = ['distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure']
features = ['shot_location_x', 'shot_location_y','distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure']
target = ['goal']

In [13]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)

In [14]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [15]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)

In [16]:
from pyspark.ml import Pipeline

# Create the pipeline
pipeline = Pipeline(stages=[log_reg])

In [17]:
xg_model = pipeline.fit(train_data)

24/12/22 00:52:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [18]:
# Make predictions
predictions = xg_model.transform(test_data)

# Evaluate using BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.7734948322466917


In [19]:
predictions.show(truncate=False)

+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+----+------------------------------------+----------------------------------------+------------------------------------------+----------+
|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_statsbomb_xg|goal|features_vector                     |rawPrediction                           |probability                               |prediction|
+---------------+---------------+----------------+----------+-------------------+--------------+-----------------+----+------------------------------------+----------------------------------------+------------------------------------------+----------+
|40.3           |44.4           |79.8214         |5.7289352 |1                  |0             |0.005853886      |0   |[79.8214,5.728935241699219,1.0,0.0] |[9.226487592666686,-9.226487592666686]  |[0.9999016114365803,9.838856341970459E-5] |0.0 

In [20]:
from pyspark.sql.functions import col, udf, format_number, expr
from pyspark.sql.types import DoubleType

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference").show(50,False)

+-----------------+----------------+-------------+
|shot_statsbomb_xg|goal_probability|difference   |
+-----------------+----------------+-------------+
|0.005853886      |0.0000983886    |0.0057554974 |
|0.01531066       |0.0000809785    |0.0152296815 |
|0.0033393882     |0.0002795487    |0.0030598395 |
|0.0044208374     |0.0003204872    |0.0041003502 |
|0.010609539      |0.0001853839    |0.0104241551 |
|0.0038471406     |0.0006122157    |0.0032349249 |
|0.0047861827     |0.0001906481    |0.0045955346 |
|0.0069873757     |0.0008136981    |0.0061736776 |
|0.0036706391     |0.0010583564    |0.0026122827 |
|0.0076118503     |0.0008650748    |0.0067467755 |
|0.0042117466     |0.0004723550    |0.0037393916 |
|0.003187881      |0.0005063547    |0.0026815263 |
|0.00458611       |0.0011737962    |0.0034123138 |
|0.003606883      |0.0012197901    |0.0023870929 |
|0.0053315936     |0.0018616301    |0.0034699635 |
|0.019038264      |0.0030376060    |0.0160006580 |
|0.0075572664     |0.0038521140

In [21]:
# average of the difference
from pyspark.sql.functions import avg

avg_diff = predictions_with_goal_prob.select(avg(col("shot_statsbomb_xg") - col("goal_probability"))).collect()[0][0]
print(f"Average difference: {avg_diff}")

Average difference: -0.0017414274527208613


In [22]:
# since the average difference is -0.0017309433278151149 which is very close to 0, we can say that the model is good

In [23]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

from pyspark.sql.functions import round

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

predictions_with_goal_prob.select("goal", "goal_probability_rounded", "shot_statsbomb_xg_rounded").show(50,False)

+----+------------------------+-------------------------+
|goal|goal_probability_rounded|shot_statsbomb_xg_rounded|
+----+------------------------+-------------------------+
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0                     |0.0                      |
|0   |0.0     

In [24]:
predictions_with_goal_prob.count()

12202

In [25]:
predictions_with_goal_prob.groupBy("goal").count().show()

+----+-----+
|goal|count|
+----+-----+
|   1| 1318|
|   0|10884|
+----+-----+



In [26]:
from pyspark.sql.functions import col, when

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0)
).withColumn(
    "statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)


In [27]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")


Model Accuracy: 89.61%
StatsBomb xG Accuracy: 90.72%


In [28]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

distance_to_goal: -0.11040576581106461
shot_angle: 0.021568386918724706
preferred_foot_shot: 0.8942180883537789
under_pressure: -0.5491043131880943
